In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('timeseries_data.csv')

In [3]:
# Filter the DataFrame to exclude rows with the year 2023
df_train = df[df['Season'] != '2023-08-01']

# Filter data for the 2023 season for testing
df_test = df[df['Season'] == '2023-08-01']

In [5]:
# Select features and target variable
features = ['GS', 'A', 'CS', 'BPS', 'MP']
target = 'Pts'

# Split the data into features (X) and target variable (y)
X = df_train[features]
y = df_train[target]

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)

# Use the scaler trained on the training set to transform the 2023 season data
X_test_scaled = scaler.transform(df_test[features])

# Create a Linear Regression model
model = LinearRegression()

# Train the model on historical data
model.fit(X_train_scaled, y)

# Make predictions on the 2023 season data
y_pred_2023 = model.predict(X_test_scaled)

# Actual target values for the 2023 season
y_actual_2023 = df_test[target]


In [6]:
mse = mean_squared_error(y_actual_2023, y_pred_2023)
r2 = r2_score(y_actual_2023, y_pred_2023)

# Display evaluation metrics
print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error: 13.5262
R-squared: 0.9727


In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
# Create a DataFrame to display the results for the 2023 season
results_2023_df = pd.DataFrame({
    'Player Name': df_test['Player Name'],
    'Team': df_test['Team'],
    'Actual Pts': df_test['Pts'],
    'Predicted Pts': y_pred_2023.round().astype(int) # Use rounded and integer predicted points
})

# Reset the index of the DataFrame
results_2023_df = results_2023_df.reset_index(drop=True)

# Display the results for the 2023 season
results_2023_df

,Player Name,Team,Actual Pts,Predicted Pts
0,Salah,LIV,127,118
1,Son,TOT,113,105
2,Haaland,MCI,112,126
3,Watkins,AVL,99,104
4,Saka,ARS,94,94
5,Mbeumo,BRE,87,78
6,Bowen,WHU,86,74
7,Gordon,NEW,83,85
8,Trippier,NEW,82,82
9,J.Alvarez,MCI,80,91


In [9]:
# Reset the options to their default values if needed
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')